In [ ]:
import requests
import zipfile
import os

# Function to download and extract a dataset
def download_and_extract(url, target_path, extract_path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(target_path, 'wb') as file:
            file.write(response.content)
        with zipfile.ZipFile(target_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)

# URLs for the datasets
NLCD_url = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/biota_landcover_nlcd_mn_2019/tif_biota_landcover_nlcd_mn_2019.zip'
DEM_url = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/elev_30m_digital_elevation_model/fgdb_elev_30m_digital_elevation_model.zip'
county_url = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/bdry_counties_in_minnesota/shp_bdry_counties_in_minnesota.zip'

# Directory where you want to save the datasets
base_dir = r"C:\Users\jake1\OneDrive\Documents\ArcGIS\Projects\Lab2_pt2"
# Create new directories for unzipped data
unzipped_NLCD_dir = os.path.join(base_dir, 'Unzipped Landcover')
unzipped_DEM_dir = os.path.join(base_dir, 'Unzipped Elevation')
unzipped_county_dir = os.path.join(base_dir, 'Unzipped Counties')

# Download and extract the landcover dataset
NLCD_path = os.path.join(base_dir, 'landcover.zip')
download_and_extract(NLCD_url, NLCD_path, unzipped_NLCD_dir)

# Download and extract the elevation dataset
DEM_path = os.path.join(base_dir, 'elevation.zip')
download_and_extract(DEM_url, DEM_path, unzipped_DEM_dir)

# Download and extract the counties dataset
county_path = os.path.join(base_dir, 'counties.zip')
download_and_extract(county_url, county_path, unzipped_county_dir)

print("All folders successfully downloaded and extracted")

In [ ]:
import os
import shutil

# Define the new folder name
merged_fh = 'geodatabase'

# Create a directory for the merged data
merged_folder = os.path.join(base_dir, merged_fh)
os.makedirs(merged_folder, exist_ok=True)

# List the subdirectories to merge
subdirectories = ['Unzipped Landcover', 'Unzipped Elevation', 'Unzipped Counties']

# Iterate through the subdirectories and copy their contents to the merged folder
for subdirectory in subdirectories:
    subdirectory_path = os.path.join(base_dir, subdirectory)
    if os.path.exists(subdirectory_path):
        for root, dirs, files in os.walk(subdirectory_path):
            for file in files:
                file_path = os.path.join(root, file)
                shutil.copy(file_path, merged_folder)
    print("Data from", subdirectory, "was merged")

### File Setup Complete!

Now, our input data are conveniently merged into a single folder called 'merged_data'

**Action Required:** Move the 'mn_county_boundaries' shape files from merged_data dir to the parent directory, Lab2_pt2.

In [ ]:
import arcpy
# Set the workspace and input feature class
arcpy.env.workspace = r"C:\Users\jake1\OneDrive\Documents\ArcGIS\Projects\Lab_Part2"
input_feature_class = "mn_county_boundaries"

# Select county boundaries
selected = arcpy.management.SelectLayerByAttribute(input_feature_class, "NEW_SELECTION", where_clause = "CTY_Name IN ('Wabasha', 'Winona', 'Olmsted')")

# Create a feature layer with the selection
arcpy.management.CopyFeatures(selected, "Selected_Counties")

# Specify the output feature class for the selected features
output_feature_class = r"C:\Users\jake1\OneDrive\Documents\ArcGIS\Projects\Lab_Part2\mn_county_boundaries_Clip"

print("Successfully Clipped counties of interest.")

### ArcGIS Pro Map 

You should now see that two feature layers just appeared in your Contents Pane: 'Selected_Counties' and 'mn_county_boundaries_Layer2'.

Remove 'mn_county_boundaries_Layer2' so you can see only the Hennepin County boundaries.

Now that we have our boundaries for Hennepin County, we need to use the Extract by Mask function to extract our land cover data to *just* our clipped county polygons.

**Action Required:** Move the "NLCD_2019_Land_Cover.tif" files to the parent directory

In [ ]:
# Confine land cover data to selected counties using extract by mask tool
land_cover = arcpy.sa.ExtractByMask(
    in_raster="NLCD_2019_Land_Cover.tif",
    in_mask_data="Selected_Counties",
    extraction_area="INSIDE",
    analysis_extent = "NLCD_2019_Land_Cover.tif"
)
print('Land Cover layer successfully extracted')

### Map Update

If correct, you should see the NLCD 2019 land cover data for Hennepin County in your contents pane as a raster layer named 'land_cover'. To see this on your map, unselect the 'Selected_Counties' layer.

Now, we need to add our DEM to the map. 

In [ ]:
# Set the directory of the DEM .gdb to the workspace
arcpy.env.workspace = r'C:\Users\jake1\OneDrive\Documents\ArcGIS\Projects\GIS_5571_Project\Unzipped Elevation\elev_30m_digital_elevation_model.gdb'
# Name of DEM file: 'digital_elevation_model_30m'

# Use extract by mask function to extract the DEM inside Hennepin County 
DEM = arcpy.sa.ExtractByMask(
    in_raster = 'digital_elevation_model_30m',
    in_mask_data = 'Selected_Counties',
    extraction_area = 'INSIDE',
    analysis_extent = 'digital_elevation_model_30m'
)

print('DEM successfully extracted')

### Map Update

If correct, you should see the DEM data for Hennepin County in your contents pane as a geodatabase raster named 'DEM'.

Now, we need to adjust the slope of our DEM.

In [ ]:
gdb_dir = (base_dir + '\Lab2_pt2.gdb')

arcpy.env.workspace = gdb_dir

# Find slope from DEM
Slope_Extrac1 = arcpy.sa.Slope(
    in_raster="Extract_digi1",
    output_measurement="DEGREE",
    z_factor=1,
    method="PLANAR",
    z_unit="METER")

print('Slope successfully adjusted')

In [ ]:
# Create start and end points
arcpy.management.CreateFeatureclass(
    out_path = gdb_dir,
    out_name = "start_end_Locations",
    geometry_type = "POINT",
    template = None,
    has_m = "DISABLED",
    has_z = "DISABLED",
    spatial_reference='PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]];-5120900 -9998100 10000;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision',
    config_keyword="",
    spatial_grid_1 = 0,
    spatial_grid_2 = 0,
    spatial_grid_3 = 0,
    out_alias = ""
)
print("Dory's house and picnic area points successfully added")

In [ ]:
# Math to figure out my slope reclassification values
def create_eql_classes(length, max_value):
    if length <= 0:
        return []  # Return an empty list if the length is non-positive

    # Calculate the value to be assigned to each element
    value = max_value / length

    # Create the list using a list comprehension
    result_list = [value * (i + 1) for i in range(length)]
    
    return result_list

# Initialize parameters
length = 5
max_value = 79.383194
# Call function
my_list = create_eql_classes(length, max_value)

# List output values
for i in my_list:
    print('value: {:.5f}'.format(i))

In [ ]:
# Reclassify DEM slope
slope_reclass = arcpy.sa.Reclassify(
    in_raster="Slope_Extrac1",
    reclass_field="VALUE",
    remap="0 16 1;16 32 2;32 48 3;48 64 4;64 79.383194 5",
    missing_values="DATA"
)
print('Slope reclassified')

In [ ]:
# Reclassify NLCD data
reclass_NLCD = arcpy.sa.Reclassify(
    in_raster="land_cover",
    reclass_field="NLCD_Land",
    remap="'Open Water' 5;'Developed, Open Space' 1;'Developed, Low Intensity' 1;'Developed, Medium Intensity' 1;'Developed, High Intensity' 5;'Barren Land' 1;'Deciduous Forest' 1;'Evergreen Forest' 1;'Mixed Forest' 1;Shrub/Scrub 1;Herbaceous 1;Hay/Pasture 3;'Cultivated Crops' 5;'Woody Wetlands' 3;'Emergent Herbaceous Wetlands' 2",
    missing_values="DATA"
)
print('Land Cover reclassified')

In [ ]:
import numpy as np

# Create Several Cost Surfaces to Test Different Model Weights
for i in np.arange(0.1, 1.0, 0.1):
    # Set Weights
    slope_weight = round(i, 1)
    landcover_weight = round((1 - i), 1)
    
    # Calculate Cost and Save as New Raster
    cost = ((((arcpy.Raster('Reclass_Slop2') * slope_weight) + (arcpy.Raster('Reclass_Extr1') * landcover_weight)) * -1) + 6)

    output_name = f"Slope_{slope_weight}_Landcover_{int(1 - landcover_weight)}"
    
    # Save cost paths to .gdb with desecriptive name
    cost.save(os.path.join(gdb_dir, output_name))

In [ ]:
import numpy as np

# Create Several Cost Surfaces to Test Different Model Weights
for i in np.arange(0.1, 1.0, 0.1):
    # Set Weights
    slope_weight = round(i, 1)
    landcover_weight = round((1 - i), 1)
    
    arcpy.sa.OptimalRegionConnections("start_end_locations", 
        fr"cPath_{str(slope_weight)[2:3]}s_{str(landcover_weight)[2:3]}lc", 
        in_cost_raster = ((((arcpy.Raster('slope_reclass') * slope_weight) + (arcpy.Raster('reclass_NLCD') * landcover_weight)) * -1) + 6)
    )

In [ ]:
# Set the workspace and environment settings
arcpy.env.workspace = gdb_dir
arcpy.env.overwriteOutput = True

# Define the input data (Land Cover and DEM)
land_cover = "reclass_NLCD"
dem = "slope_reclass"

# Define the output folder for saving results
output_folder = base_dir

# Create a list of weight values to iterate through
weight_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# Iterate through different weight values
for weight in weight_values:
    # Calculate suitability using weighted overlay
    weighted_suitability = (arcpy.Raster(land_cover) * weight) + (arcpy.Raster(dem) * (1 - weight))

    # Normalize the suitability scores if needed
    # normalized_suitability = arcpy.Normalize(weighted_suitability)

    # Save the suitability raster with a filename based on the weight
    output_name = f"Suitability_w{int(weight * 100)}.tif"
    output_path = os.path.join(output_folder, output_name)
    weighted_suitability.save(output_path)

    # Optionally, perform further analysis or reporting here

print("Sensitivity analysis completed.")

In [ ]:
# Optional routing function returned error... I'll just show suitability analysis

# Analysis of Suitability 

# Set your workspace folder where output files are located
arcpy.env.workspace = base_dir
workspace = base_dir

# List of output raster files
output_files = [os.path.join(workspace, filename) for filename in os.listdir(workspace) if filename.endswith('.tif')]

# Analyze each output file
for file_path in output_files:
    # Example analysis tasks (you can customize this)
    
    # Open the raster file
    raster = arcpy.Raster(file_path)  # Use arcpy or other libraries for different GIS formats
    
    # Calculate statistics
    mean = arcpy.GetRasterProperties_management(raster, "MEAN")
    min_value = arcpy.GetRasterProperties_management(raster, "MINIMUM")
    max_value = arcpy.GetRasterProperties_management(raster, "MAXIMUM")
    
    # Print or save the results
    print(f"Analysis for {file_path}:")
    print(f"Mean: {mean}")
    print(f"Minimum: {min_value}")
    print(f"Maximum: {max_value}")

In [ ]:
import os
import pandas as pd

# List of output raster files
output_files = [os.path.join(workspace, filename) for filename in os.listdir(workspace) if filename.endswith('.tif')]

# Define the criteria names
criteria = ["Land Cover", "Slope"]

# Create an empty DataFrame to store the statistics
statistics_df = pd.DataFrame(columns=["File Name", "Criteria and Weight", "Mean", "Minimum", "Maximum"])

# Calculate the number of output files (excluding the first one)
num_files = len(output_files) - 1

# Analyze each output file (ignoring the first) and append statistics to the DataFrame
for i, file_path in enumerate(output_files[1:], start=1):
    # Example analysis tasks (you can customize this)
    
    # Open the raster file
    raster = arcpy.Raster(file_path)  # Use arcpy or other libraries for different GIS formats
    
    # Calculate statistics
    mean = arcpy.GetRasterProperties_management(raster, "MEAN")
    min_value = arcpy.GetRasterProperties_management(raster, "MINIMUM")
    max_value = arcpy.GetRasterProperties_management(raster, "MAXIMUM")
    
    # Extract the file name from the path
    file_name = os.path.basename(file_path)
    
    # Calculate weights based on linear progression, with 0.5 for the middle value
    weight_land_cover = i / num_files if i <= num_files / 2 else 1 - (i - num_files / 2) / num_files
    weight_slope = 1 - weight_land_cover
    
    # Create a single string for Criteria and Weight
    criteria_and_weight = f"{criteria[0]} Weight {weight_land_cover:.1f}, {criteria[1]} Weight {weight_slope:.1f}"
    
    # Append statistics to the DataFrame
    statistics_df = statistics_df.append({"File Name": file_name, "Criteria and Weight": criteria_and_weight, "Mean": mean, "Minimum": min_value, "Maximum": max_value}, ignore_index=True)

# Save the table to a CSV file
output_table = r"C:\Users\jake1\OneDrive\Documents\ArcGIS\Projects\Lab2_pt2\results_table.csv"
statistics_df.to_csv(output_table, index=False)

print('Table successfully created')